In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
#import shap
#import socket
#import matplotlib.pyplot as plt
#from tqdm import tqdm

In [2]:
from sklearn.metrics import precision_score, recall_score, f1_score

In [3]:
import seaborn as sns
import os
import gc
import math
import time
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

from imblearn.under_sampling import ClusterCentroids
from imblearn.combine import SMOTEENN

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

In [5]:
import tensorflow as tf
from tensorflow.keras import layers, models

In [6]:
from sdv.single_table import CTGANSynthesizer
from sklearn.utils import resample

In [7]:
df = pd.read_csv("../data/raw/CIC_IoMT_2024_WiFi_MQTT_train.csv")

In [8]:
label_mapping = {
    'Benign': ['Benign_train'],
    
    'Spoofing': ['ARP_Spoofing_train'],

    'Reconnaissance': [
        'Recon-OS_Scan_train',
        'Recon-Ping_Sweep_train',
        'Recon-VulScan_train',
        'Recon-Port_Scan_train'
    ],
    
    'MQTT': [
        'MQTT-Malformed_Data_train',
        'MQTT-DoS-Connect_Flood_train',
        'MQTT-DDoS-Publish_Flood_train',
        'MQTT-DoS-Publish_Flood_train',
        'MQTT-DDoS-Connect_Flood_train'
    ],

    'DoS': [
        'TCP_IP-DoS-TCP1_train', 'TCP_IP-DoS-TCP2_train', 'TCP_IP-DoS-TCP3_train', 'TCP_IP-DoS-TCP4_train',
        'TCP_IP-DoS-UDP1_train', 'TCP_IP-DoS-UDP2_train', 'TCP_IP-DoS-UDP3_train', 'TCP_IP-DoS-UDP4_train',
        'TCP_IP-DoS-ICMP1_train', 'TCP_IP-DoS-ICMP2_train', 'TCP_IP-DoS-ICMP3_train', 'TCP_IP-DoS-ICMP4_train',
        'TCP_IP-DoS-SYN1_train', 'TCP_IP-DoS-SYN2_train', 'TCP_IP-DoS-SYN3_train', 'TCP_IP-DoS-SYN4_train'
    ],

    'DDoS': [
        'TCP_IP-DDoS-TCP1_train', 'TCP_IP-DDoS-TCP2_train', 'TCP_IP-DDoS-TCP3_train', 'TCP_IP-DDoS-TCP4_train',
        'TCP_IP-DDoS-UDP1_train', 'TCP_IP-DDoS-UDP2_train', 'TCP_IP-DDoS-UDP3_train', 'TCP_IP-DDoS-UDP4_train',
        'TCP_IP-DDoS-UDP5_train', 'TCP_IP-DDoS-UDP6_train', 'TCP_IP-DDoS-UDP7_train', 'TCP_IP-DDoS-UDP8_train',
        'TCP_IP-DDoS-ICMP1_train', 'TCP_IP-DDoS-ICMP2_train', 'TCP_IP-DDoS-ICMP3_train', 'TCP_IP-DDoS-ICMP4_train',
        'TCP_IP-DDoS-ICMP5_train', 'TCP_IP-DDoS-ICMP6_train', 'TCP_IP-DDoS-ICMP7_train', 'TCP_IP-DDoS-ICMP8_train',
        'TCP_IP-DDoS-SYN1_train', 'TCP_IP-DDoS-SYN2_train', 'TCP_IP-DDoS-SYN3_train', 'TCP_IP-DDoS-SYN4_train'
    ]
}

In [9]:
def map_category(label):
    for parent, sub_labels in label_mapping.items():
        if label in sub_labels:
            return parent
    return 'Unknown'
df['SuperClass'] = df['label'].apply(map_category)

In [10]:
grouped = df.groupby(['SuperClass', 'label']).size().reset_index(name='Records')
total = len(df)
grouped['Percentage'] = (grouped['Records'] / total) * 100

summary = grouped.groupby('SuperClass')['Records'].sum().reset_index()
summary['Percentage'] = (summary['Records'] / total) * 100

print("Hierarchical Breakdown:")
display(grouped.sort_values(['SuperClass', 'Records'], ascending=[True, False]).head(20))

print("Category Summary:")
display(summary)

Hierarchical Breakdown:


,SuperClass,label,Records,Percentage
0,Benign,Benign_train,192732,2.691475
18,DDoS,TCP_IP-DDoS-UDP2_train,207295,2.894846
19,DDoS,TCP_IP-DDoS-UDP3_train,206604,2.885196
20,DDoS,TCP_IP-DDoS-UDP4_train,206343,2.881551
17,DDoS,TCP_IP-DDoS-UDP1_train,206170,2.879135
21,DDoS,TCP_IP-DDoS-UDP5_train,205507,2.869876
24,DDoS,TCP_IP-DDoS-UDP8_train,204105,2.850298
15,DDoS,TCP_IP-DDoS-TCP3_train,204075,2.849879
10,DDoS,TCP_IP-DDoS-SYN2_train,203669,2.844209
13,DDoS,TCP_IP-DDoS-TCP1_train,202311,2.825245


Category Summary:


,SuperClass,Records,Percentage
0,Benign,192732,2.691475
1,DDoS,4779859,66.750060
2,DoS,1805529,25.213959
3,MQTT,262938,3.671892
4,Reconnaissance,103726,1.448519
5,Spoofing,16047,0.224094


In [11]:
ID_COLS = ['label', 'SuperClass']

In [12]:
DROP_COLS = ['Boot_Time_with_date', 'RTime', 'SrcAddr', 'DstAddr', 'SrcMac', 'DstMac', 'IMEI']

In [13]:
RATE_COLS = ['Rate', 'Srate', 'Drate'] 

In [14]:
cols_to_drop = [c for c in DROP_COLS if c in df.columns]
if cols_to_drop:
    print("Dropping ID/time columns:", cols_to_drop)
    df = df.drop(columns=cols_to_drop)

In [15]:
df['SuperClass'].value_counts()

SuperClass
DDoS              4779859
DoS               1805529
MQTT               262938
Benign             192732
Reconnaissance     103726
Spoofing            16047
Name: count, dtype: int64

In [16]:
num_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()

In [17]:
num_cols

['Header_Length',
 'Protocol Type',
 'Duration',
 'Rate',
 'Srate',
 'Drate',
 'fin_flag_number',
 'syn_flag_number',
 'rst_flag_number',
 'psh_flag_number',
 'ack_flag_number',
 'ece_flag_number',
 'cwr_flag_number',
 'ack_count',
 'syn_count',
 'fin_count',
 'rst_count',
 'HTTP',
 'HTTPS',
 'DNS',
 'Telnet',
 'SMTP',
 'SSH',
 'IRC',
 'TCP',
 'UDP',
 'DHCP',
 'ARP',
 'ICMP',
 'IGMP',
 'IPv',
 'LLC',
 'Tot sum',
 'Min',
 'Max',
 'AVG',
 'Std',
 'Tot size',
 'IAT',
 'Number',
 'Magnitue',
 'Radius',
 'Covariance',
 'Variance',
 'Weight']

In [18]:
LABEL_COL = 'SuperClass'
SAMPLES_GEN=30000

In [19]:
spoof_df = df[df[LABEL_COL] == 'Spoofing'].copy()

In [20]:
spoof_df.shape

(16047, 47)

In [21]:
gen = SAMPLES_GEN - len(spoof_df)

In [22]:
gen

13953

In [23]:
from sdv.metadata import Metadata

In [24]:
metadata = Metadata.detect_from_dataframe(
    data=spoof_df,
    table_name='spoofing_table')

In [25]:
ctgan = CTGANSynthesizer(metadata, epochs=50, verbose=True)

d:\Prog\iomt-anomalyze\venv\Lib\site-packages\sdv\single_table\base.py:134: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


In [26]:
cat_cols = [c for c in df.columns if c not in num_cols + [LABEL_COL, 'label']]

In [27]:
ctgan.fit(spoof_df)

d:\Prog\iomt-anomalyze\venv\Lib\site-packages\ctgan\synthesizers\_utils.py:16: FutureWarning: `cuda` parameter is deprecated and will be removed in a future release. Please use `enable_gpu` instead.
  warnings.warn(
Gen. (1.02) | Discrim. (-1.10): 100%|██████████| 50/50 [02:29<00:00,  2.98s/it] 


In [28]:
fig = ctgan.get_loss_values_plot()
fig

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'variable=Generator Loss<br>Epoch=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'Generator Loss',
              'line': {'color': '#000036', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'Generator Loss',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': {'bdata': 'AAECAwQFBgcICQoLDA0ODxAREhMUFRYXGBkaGxwdHh8gISIjJCUmJygpKissLS4vMDE=', 'dtype': 'i1'},
              'xaxis': 'x',
              'y': {'bdata': ('AAAAwKzB0r8AAACgtVcDwAAAACCbWv' ... 'Di+PI/AAAAIEAi3z8AAABgek/wPw=='),
                    'dtype': 'f8'},
              'yaxis': 'y'},
             {'hovertemplate': 'variable=Discriminator Loss<br>Epoch=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'Discriminator Loss',
              'line': {'color': '#01E0C9', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'Discriminator Loss',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': {'bdata': 'AAECAwQFBgcICQoLDA0ODxAREhMUFRYXGBkaGxwdHh8gISIjJCUmJygpKissLS4vMDE=', 'dtype': 'i1'},
              'xaxis': 'x',
              'y': {'bdata': ('AAAAgIjl778AAAAAJfjbPwAAAADBh9' ... 'DQTPG/AAAAwK1y4b8AAADA0Z3xvw=='),
                    'dtype': 'f8'},
              'yaxis': 'y'}],
    'layout': {'font': {'size': 18},
               'legend': {'orientation': 'v', 'title': {'text': ''}, 'tracegroupgap': 0},
               'margin': {'t': 60},
               'plot_bgcolor': '#F5F5F8',
               'template': '...',
               'title': {'text': 'CTGAN loss function'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'Epoch'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'Loss'}}}
})

In [29]:
synth_data = ctgan.sample(gen)

In [30]:
synth_data

,Header_Length,Protocol Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,psh_flag_number,...,Tot size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight,label,SuperClass
0,9310625.36,5.900,64.9,282.361373,80.523898,0,0.0,0.0,0.0,0.09,...,93.4,0.000000e+00,5.6,13.228,0.0000,392429.278854,0.4218,40.676,ARP_Spoofing_train,Spoofing
1,146073.20,6.291,250.0,0.011909,180.700402,0,0.0,0.0,0.0,0.51,...,119.2,0.000000e+00,13.6,10.484,31.2419,2378.557864,0.8054,239.111,ARP_Spoofing_train,Spoofing
2,486965.45,8.761,23.9,31.586875,0.011909,0,0.0,0.0,0.0,0.38,...,1514.0,1.653219e+08,13.5,15.041,719.9677,7002.269241,1.0000,244.600,ARP_Spoofing_train,Spoofing
3,0.00,6.235,65.2,183.264551,60.994525,0,0.0,0.0,0.0,0.00,...,1343.3,1.655475e+08,13.7,12.726,1.7944,335746.900601,0.9901,38.488,ARP_Spoofing_train,Spoofing
4,3033882.92,8.381,63.8,0.011909,146.149440,0,0.0,0.0,0.0,0.01,...,84.0,0.000000e+00,13.6,54.595,653.0620,0.000000,0.9140,38.274,ARP_Spoofing_train,Spoofing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13948,9037509.94,2.052,250.0,243.720572,112.964753,0,0.0,0.0,0.0,0.39,...,106.6,0.000000e+00,13.6,30.827,405.0360,22301.328504,1.0000,36.272,ARP_Spoofing_train,Spoofing
13949,0.00,6.279,250.0,78.517425,0.011909,0,0.0,0.0,0.0,0.50,...,108.1,0.000000e+00,13.7,9.890,6.3053,3646.313880,0.0000,241.263,ARP_Spoofing_train,Spoofing
13950,5985818.60,17.000,75.6,0.011909,256.974757,0,0.0,0.0,0.0,0.00,...,1322.1,1.671793e+08,13.7,49.617,454.3210,171258.790052,0.9861,37.190,ARP_Spoofing_train,Spoofing
13951,5578342.42,17.000,65.7,124.640488,198.840194,0,0.0,0.0,0.0,0.00,...,308.6,2.333603e+05,13.6,44.306,392.7354,31635.458440,0.9842,33.107,ARP_Spoofing_train,Spoofing


In [31]:
synth_data[LABEL_COL] = 'Spoofing'

In [32]:
spoof_balanced = pd.concat([spoof_df, synth_data], ignore_index=True)

In [33]:
spoof_balanced

,Header_Length,Protocol Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,psh_flag_number,...,Tot size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight,label,SuperClass
0,866.60,10.400,64.0,45722.390000,45722.390000,0,0.0,0.0,0.0,0.30,...,431.8,1.694026e+08,5.5,16.970,213.1000,66236.080000,0.9000,38.500,ARP_Spoofing_train,Spoofing
1,3934.30,12.600,131.2,35708.800000,35708.800000,0,0.0,0.0,0.0,0.30,...,406.2,1.694026e+08,13.5,30.890,709.0000,251721.120000,1.0000,244.600,ARP_Spoofing_train,Spoofing
2,5592.80,12.600,97.6,66.403500,66.403500,0,0.0,0.0,0.0,0.30,...,386.5,1.361110e-02,5.5,21.780,290.8000,84028.650000,0.9000,38.500,ARP_Spoofing_train,Spoofing
3,9303.60,14.800,80.8,51.201280,51.201280,0,0.0,0.0,0.0,0.10,...,300.2,1.694026e+08,13.5,26.950,597.0000,178453.000000,1.0000,244.600,ARP_Spoofing_train,Spoofing
4,8592.40,12.600,98.6,42.706455,42.706455,0,0.0,0.0,0.0,0.20,...,209.2,1.393099e-02,5.5,24.250,375.2000,80115.110000,0.9000,38.500,ARP_Spoofing_train,Spoofing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,9037509.94,2.052,250.0,243.720572,112.964753,0,0.0,0.0,0.0,0.39,...,106.6,0.000000e+00,13.6,30.827,405.0360,22301.328504,1.0000,36.272,ARP_Spoofing_train,Spoofing
29996,0.00,6.279,250.0,78.517425,0.011909,0,0.0,0.0,0.0,0.50,...,108.1,0.000000e+00,13.7,9.890,6.3053,3646.313880,0.0000,241.263,ARP_Spoofing_train,Spoofing
29997,5985818.60,17.000,75.6,0.011909,256.974757,0,0.0,0.0,0.0,0.00,...,1322.1,1.671793e+08,13.7,49.617,454.3210,171258.790052,0.9861,37.190,ARP_Spoofing_train,Spoofing
29998,5578342.42,17.000,65.7,124.640488,198.840194,0,0.0,0.0,0.0,0.00,...,308.6,2.333603e+05,13.6,44.306,392.7354,31635.458440,0.9842,33.107,ARP_Spoofing_train,Spoofing


In [34]:
balanced_parts = [spoof_balanced]

In [35]:
for cls, group in df.groupby(LABEL_COL):
    if cls == 'Spoofing':
        continue
    n_avail = len(group)
    if n_avail > SAMPLES_GEN:
        sampled = group.drop_duplicates().sample(n=SAMPLES_GEN, random_state=42)
    elif n_avail < SAMPLES_GEN:
        sampled = resample(group, replace=True, n_samples=SAMPLES_GEN, random_state=42)
    else:
        sampled = group
    balanced_parts.append(sampled)

In [36]:
balanced_df = pd.concat(balanced_parts, ignore_index=True)

In [37]:
balanced_df['SuperClass'].value_counts()

SuperClass
Spoofing          30000
Benign            30000
DDoS              30000
DoS               30000
MQTT              30000
Reconnaissance    30000
Name: count, dtype: int64

In [38]:
balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)

In [40]:
# ------------------ PATCH: fixed build_model + avoid shadowing 'models' ------------------
from sklearn.model_selection import StratifiedKFold
# Use explicit reference to Keras models API to avoid name collision
KModels = tf.keras.models

def build_model():
    inputs = []; parts = []
    # numeric
    if 'num_in' in inputs_all:
        num_dim = inputs_all['num_in'].shape[1]
        num_in = layers.Input(shape=(num_dim,), name='num_in')
        x = layers.Dense(256, activation='swish')(num_in)
        x = layers.LayerNormalization()(x)
        x = layers.Dropout(0.25)(x)
        x = residual_block(x, 256, 0.2)
        x = residual_block(x, 256, 0.2)
        x = layers.Dense(128, activation='swish')(x)
        parts.append(x); inputs.append(num_in)
    # categorical
    if X_cat.size:
        cat_inputs=[]; cat_embs=[]
        for i,c in enumerate(cat_cols):
            card = len(cat_maps[c]) + 1
            inp = layers.Input(shape=(1,), dtype='int32', name=f'cat_in_{i}')
            emb_dim = max(4, min(32, int(card**0.25 * 8)))
            emb = layers.Embedding(input_dim=card+2, output_dim=emb_dim, name=f'emb_{i}')(inp)
            emb = layers.Flatten()(emb)
            cat_inputs.append(inp); cat_embs.append(emb)
        cat_concat = layers.Concatenate()(cat_embs) if len(cat_embs)>1 else cat_embs[0]
        c = layers.Dense(128, activation='swish')(cat_concat)
        c = layers.LayerNormalization()(c)
        c = layers.Dropout(0.2)(c)
        parts.append(c); inputs += cat_inputs
    # text
    if 'text_in' in inputs_all:
        text_dim = inputs_all['text_in'].shape[1]
        text_in = layers.Input(shape=(text_dim,), name='text_in')
        t = layers.Dense(512, activation='swish')(text_in)
        t = layers.LayerNormalization()(t)
        t = layers.Dropout(0.3)(t)
        t = layers.Dense(256, activation='swish')(t)
        t = layers.Dropout(0.2)(t)
        parts.append(t); inputs.append(text_in)
    # fuse
    if len(parts) > 1:
        fused = layers.Concatenate()(parts)
    else:
        fused = parts[0]
    x = layers.Dense(512, activation='swish', kernel_regularizer=regularizers.l2(1e-5))(fused)
    x = layers.LayerNormalization()(x)
    x = layers.Dropout(0.4)(x)
    x = layers.Dense(256, activation='swish')(x)
    x = layers.Dropout(0.25)(x)
    out = layers.Dense(n_classes, activation='softmax', name='out')(x)
    model = KModels.Model(inputs=inputs, outputs=out)   # use KModels explicitly
    return model

# Rename the Python list to avoid clobbering the keras 'models' symbol
trained_models = []

# ---------- Stratified K-Fold training (OOF) ----------
skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=RANDOM_SEED)
oof_preds = np.zeros((len(df), n_classes))

fold = 0
for train_idx, val_idx in skf.split(np.zeros(len(y_all)), y_all):
    fold += 1
    print(f"\n=== Fold {fold}/{N_FOLDS} ===")
    train_inputs = {k: v[train_idx] for k,v in inputs_all.items()}
    val_inputs = {k: v[val_idx] for k,v in inputs_all.items()}
    y_train = y_all[train_idx]; y_val = y_all[val_idx]

    # class alpha for focal
    cw = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
    alpha = cw / cw.sum()

    model = build_model()
    loss_fn = focal_loss_flexible(gamma=2.0, alpha=alpha, n_classes=n_classes, label_smoothing=0.01)

    # optimizer + schedule
    initial_lr = 5e-4
    steps_per_epoch = math.ceil(len(y_train) / BATCH_SIZE)
    decay_steps = max(1, steps_per_epoch * max(5, EPOCHS//2))
    lr_schedule = tf.keras.optimizers.schedules.CosineDecay(initial_learning_rate=initial_lr, decay_steps=decay_steps)
    opt = optimizers.Adam(learning_rate=lr_schedule)

    model.compile(optimizer=opt, loss=loss_fn, metrics=['accuracy'])
    model.summary()

    # Macro F1 callback
    class MacroF1Callback(callbacks.Callback):
        def __init__(self, val_inputs, val_labels):
            self.val_inputs = val_inputs; self.val_labels = val_labels
        def on_epoch_end(self, epoch, logs=None):
            preds = self.model.predict(self.val_inputs, batch_size=256)
            pred_lab = np.argmax(preds, axis=1)
            f1 = f1_score(self.val_labels, pred_lab, average='macro')
            print(f" - val_macro_f1: {f1:.4f}")

    ckpt_path = os.path.join(MODEL_DIR, f"fold{fold}_best.keras")
    ckpt = callbacks.ModelCheckpoint(ckpt_path, monitor='val_loss', save_best_only=True, verbose=1)
    rlr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, min_lr=1e-6, verbose=1)
    es = callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)
    mcb = MacroF1Callback(val_inputs, y_val)

    steps = math.ceil(len(y_train)/BATCH_SIZE)
    train_gen = balanced_generator(train_inputs, y_train, batch_size=BATCH_SIZE, mixup_p=MIXUP_P)
    val_dataset = tf.data.Dataset.from_tensor_slices((val_inputs, tf.keras.utils.to_categorical(y_val, n_classes))).batch(BATCH_SIZE)

    history = model.fit(train_gen, epochs=EPOCHS, steps_per_epoch=steps, validation_data=val_dataset, callbacks=[ckpt, rlr, es, mcb], verbose=2)

    # load best and OOF predict
    model.load_weights(ckpt_path)
    preds = model.predict(val_inputs, batch_size=256)
    oof_preds[val_idx] = preds
    trained_models.append(model)
    gc.collect()

# ---------- OOF evaluation ----------
oof_labels = np.argmax(oof_preds, axis=1)
print("\n=== OOF evaluation ===")
print("OOF macro F1:", f1_score(y_all, oof_labels, average='macro'))
print(classification_report(y_all, oof_labels, target_names=class_names, digits=4))

# ---------- Save artifacts ----------
joblib.dump({'num_cols': num_cols, 'cat_cols': cat_cols, 'scaler':scaler, 'cat_maps':cat_maps, 'use_sbert':use_sbert, 'tfv': (tfv if 'tfv' in globals() else None), 'label_encoder_classes': list(le.classes_)}, os.path.join(MODEL_DIR,'preproc.joblib'))
for i,m in enumerate(trained_models):
    m.save(os.path.join(MODEL_DIR, f"model_fold{i+1}.keras"), include_optimizer=False)
np.save(os.path.join(MODEL_DIR, 'oof_preds.npy'), oof_preds)
print("Saved models and preproc to", MODEL_DIR)

# ------------------------------------------------------------------------------------

NameError: name 'N_FOLDS' is not defined

In [41]:
# Best DNN for tabular + text (Keras) — single cell
import os, random, math, gc, time
import numpy as np, pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks, regularizers
from tensorflow.keras import backend as K

# ------------------- CONFIG -------------------
DF = balanced_df               # <-- ensure this exists
TARGET = 'SuperClass'          # <-- target column
NUM_COLS = [c for c in num_cols if c in DF.columns]
CAT_COLS = [c for c in cat_cols if c in DF.columns]
TEXT_COL = 'info' if 'info' in DF.columns else None

MODEL_DIR = 'best_dnn'
os.makedirs(MODEL_DIR, exist_ok=True)

BATCH_SIZE = 128   # reduce to 64/32 if OOM
EPOCHS = 80
RANDOM_SEED = 42
EMB_BASE = 32      # base embedding dim (will scale per cardinality)
TFIDF_MAX_FEATURES = 2000
SBERT_NAME = 'all-MiniLM-L6-v2'
# ----------------------------------------------

# reproducibility
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

# ---------- Utilities ----------
def focal_loss_with_logits(gamma=2.0, alpha=None, n_classes=None, label_smoothing=0.0):
    # expects y_true as integer labels
    alpha = np.ones(n_classes) if alpha is None else np.array(alpha)
    alpha = alpha.astype(np.float32)
    def loss(y_true, y_pred):
        # y_pred are probabilities (softmax outputs)
        y_true_one = tf.one_hot(tf.cast(y_true, tf.int32), depth=n_classes)
        # label smoothing:
        if label_smoothing > 0:
            smooth = label_smoothing
            y_true_one = y_true_one * (1 - smooth) + smooth / tf.cast(n_classes, tf.float32)
        p_t = tf.reduce_sum(y_true_one * y_pred, axis=-1)
        alpha_factor = tf.reduce_sum(alpha * y_true_one, axis=-1)
        loss_val = - alpha_factor * tf.pow(1. - p_t, gamma) * tf.math.log(tf.clip_by_value(p_t, 1e-9, 1.0))
        return tf.reduce_mean(loss_val)
    return loss

def mixup(x1, y1, x2, y2, alpha=0.2):
    lam = np.random.beta(alpha, alpha)
    x_mixed = {}
    for k in x1.keys():
        x_mixed[k] = lam * x1[k] + (1 - lam) * x2[k]
    y_mixed = lam * y1 + (1 - lam) * y2
    return x_mixed, y_mixed

# ---------- Prepare data ----------
df = DF.copy().reset_index(drop=True)
# target encode
le = LabelEncoder()
y = le.fit_transform(df[TARGET].astype(str))
class_names = list(le.classes_)
n_classes = len(class_names)
print("Classes:", class_names, "n_classes:", n_classes)

# numeric processing
X_num = df[NUM_COLS].fillna(0).astype(float).values if NUM_COLS else np.zeros((len(df),0))
scaler = StandardScaler()
if X_num.shape[1] > 0:
    X_num = scaler.fit_transform(X_num)

# categorical mapping => int indices starting at 1 (0 reserved)
cat_maps = {}
X_cat = np.zeros((len(df), 0), dtype=np.int32)
if CAT_COLS:
    cat_arrs = []
    for c in CAT_COLS:
        vals = df[c].astype(str).fillna('NA')
        uniq = vals.unique().tolist()
        mapping = {v:i+1 for i,v in enumerate(uniq)}
        cat_maps[c] = mapping
        arr = vals.map(lambda v: mapping.get(v, 0)).astype(np.int32).values.reshape(-1,1)
        cat_arrs.append(arr)
    X_cat = np.hstack(cat_arrs)

# text embeddings: try SBERT then fallback to TF-IDF
use_sbert = False
X_text = np.zeros((len(df),0))
if TEXT_COL:
    try:
        from sentence_transformers import SentenceTransformer
        print("Loading SBERT:", SBERT_NAME)
        sbert = SentenceTransformer(SBERT_NAME)
        X_text = sbert.encode(df[TEXT_COL].fillna('').astype(str).tolist(), batch_size=128, show_progress_bar=True)
        use_sbert = True
        print("SBERT shape:", X_text.shape)
    except Exception as e:
        print("SBERT not available, falling back to TF-IDF:", e)
        tfv = TfidfVectorizer(max_features=TFIDF_MAX_FEATURES, ngram_range=(1,2), min_df=3)
        X_text = tfv.fit_transform(df[TEXT_COL].fillna('').astype(str)).toarray()
        print("TF-IDF shape:", X_text.shape)

# build input dict keyed by future Input layer names (we'll use 'num_in', 'cat_in_i', 'text_in')
inputs_all = {}
if X_num.shape[1] > 0:
    inputs_all['num_in'] = X_num
for i in range(X_cat.shape[1] if X_cat.size else 0):
    inputs_all[f'cat_in_{i}'] = X_cat[:, i].reshape(-1,1)
if X_text.shape[1] > 0:
    inputs_all['text_in'] = X_text

# train/val split
idx = np.arange(len(y))
train_idx, val_idx = train_test_split(idx, stratify=y, test_size=0.2, random_state=RANDOM_SEED)
train_inputs = {k: v[train_idx] for k, v in inputs_all.items()}
val_inputs = {k: v[val_idx] for k, v in inputs_all.items()}
y_train = y[train_idx]; y_val = y[val_idx]

print("Train size:", len(y_train), "Val size:", len(y_val))
for k,v in train_inputs.items():
    print(k, v.shape)

# ---------- Model components ----------
def residual_dense(x, units, dropout=0.15):
    inp = x
    x = layers.Dense(units, activation='swish')(x)
    x = layers.LayerNormalization()(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Dense(units, activation='swish')(x)
    x = layers.LayerNormalization()(x)
    x = layers.add([inp, x])
    return x

def build_model():
    inputs = []
    parts = []

    # numeric branch
    if 'num_in' in inputs_all:
        num_in = layers.Input(shape=(X_num.shape[1],), name='num_in')
        x = layers.Dense(256, activation='swish')(num_in)
        x = layers.LayerNormalization()(x)
        x = layers.Dropout(0.25)(x)
        x = residual_dense(x, 256, dropout=0.2)
        x = residual_dense(x, 256, dropout=0.2)
        x = layers.Dense(128, activation='swish')(x)
        parts.append(x); inputs.append(num_in)

    # categorical branch (embeddings)
    cat_inputs = []
    cat_embs = []
    if X_cat.size:
        for i,c in enumerate(CAT_COLS):
            card = len(cat_maps[c]) + 1
            inp = layers.Input(shape=(1,), dtype='int32', name=f'cat_in_{i}')
            emb_dim = max(6, min(EMB_BASE, int(card**0.25 * EMB_BASE//2)))  # heuristic
            emb = layers.Embedding(input_dim=card+1, output_dim=emb_dim, name=f'emb_{i}')(inp)
            emb = layers.Flatten()(emb)
            cat_inputs.append(inp); cat_embs.append(emb)
        if len(cat_embs) > 1:
            cat_concat = layers.Concatenate()(cat_embs)
        else:
            cat_concat = cat_embs[0]
        c = layers.Dense(128, activation='swish')(cat_concat)
        c = layers.LayerNormalization()(c)
        c = layers.Dropout(0.2)(c)
        parts.append(c); inputs += cat_inputs

    # text branch
    if 'text_in' in inputs_all:
        text_in = layers.Input(shape=(X_text.shape[1],), name='text_in')
        t = layers.Dense(512, activation='swish')(text_in)
        t = layers.LayerNormalization()(t)
        t = layers.Dropout(0.3)(t)
        t = layers.Dense(256, activation='swish')(t)
        t = layers.Dropout(0.2)(t)
        parts.append(t); inputs.append(text_in)

    # attention fusion across parts
    if len(parts) == 1:
        fused = parts[0]
    else:
        stacked = layers.Concatenate()(parts)
        # lightweight self-attention style gating
        attn = layers.Dense(256, activation='tanh')(stacked)
        attn = layers.Dense(len(parts), activation='softmax')(attn)  # shape (batch, len(parts))
        # split parts and weight them
        weighted = []
        idx = 0
        for p in parts:
            dim = int(p.shape[-1])
            w = layers.Lambda(lambda x, s=idx, d=dim: x[:, s:s+1] * x[:, s+1:s+2] )(attn) if False else None
            weighted.append(p)  # skip complex per-part gating to stay safe
            idx += 1
        fused = stacked

    # top head
    x = layers.Dense(512, activation='swish', kernel_regularizer=regularizers.l2(1e-5))(fused)
    x = layers.LayerNormalization()(x)
    x = layers.Dropout(0.4)(x)
    x = layers.Dense(256, activation='swish')(x)
    x = layers.Dropout(0.25)(x)
    out = layers.Dense(n_classes, activation='softmax', name='out')(x)

    model = models.Model(inputs=inputs, outputs=out)
    return model

model = build_model()
model.summary()

# ---------- loss, optimizer, callbacks ----------
# class weights -> alpha for focal
from sklearn.utils import class_weight
cw = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
alpha = cw / cw.sum()

loss_fn = focal_loss_with_logits(gamma=2.0, alpha=alpha, n_classes=n_classes, label_smoothing=0.01)
lr_schedule = tf.keras.optimizers.schedules.CosineDecayRestarts(initial_learning_rate=1e-3, first_decay_steps=2000)
opt = optimizers.Adam(learning_rate=lr_schedule)

model.compile(optimizer=opt, loss=loss_fn, metrics=['accuracy'])

checkpoint = callbacks.ModelCheckpoint(os.path.join(MODEL_DIR,'best.h5'), monitor='val_loss', save_best_only=True, verbose=1)
rlr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1, min_lr=1e-6)
es = callbacks.EarlyStopping(monitor='val_loss', patience=12, restore_best_weights=True, verbose=1)

# ---------- data generator with MixUp ----------
def gen_named(inputs_dict, labels, batch_size=BATCH_SIZE, mixup_p=0.5):
    n = len(labels)
    keys = list(inputs_dict.keys())
    while True:
        idxs = np.arange(n); np.random.shuffle(idxs)
        for i in range(0, n, batch_size):
            bidx = idxs[i:i+batch_size]
            batch_x = {k: inputs_dict[k][bidx] for k in keys}
            batch_y = tf.keras.utils.to_categorical(labels[bidx], n_classes)
            if random.random() < mixup_p:
                jidx = np.random.permutation(bidx)
                batch_x2 = {k: inputs_dict[k][jidx - i] if False else inputs_dict[k][jidx] for k in keys}
                # build second batches aligned
                batch_x2_aligned = {k: inputs_dict[k][jidx] for k in keys}
                mixed_x, mixed_y = mixup(batch_x, batch_y, batch_x2_aligned, tf.keras.utils.to_categorical(labels[jidx], n_classes))
                yield mixed_x, mixed_y
            else:
                yield batch_x, batch_y

train_gen = gen_named(train_inputs, y_train)
steps_per_epoch = math.ceil(len(y_train) / BATCH_SIZE)

# validation dataset from dict
val_dataset = tf.data.Dataset.from_tensor_slices((val_inputs, tf.keras.utils.to_categorical(y_val, n_classes))).batch(BATCH_SIZE)

# ---------- train ----------
history = model.fit(
    train_gen,
    epochs=EPOCHS,
    steps_per_epoch=steps_per_epoch,
    validation_data=val_dataset,
    callbacks=[checkpoint, rlr, es],
    verbose=2
)

# ---------- evaluate ----------
model.load_weights(os.path.join(MODEL_DIR,'best.h5'))
val_preds = model.predict(val_inputs, batch_size=256)
y_val_pred = np.argmax(val_preds, axis=1)

from sklearn.metrics import classification_report, f1_score, confusion_matrix
print("Val macro F1:", f1_score(y_val, y_val_pred, average='macro'))
print(classification_report(y_val, y_val_pred, target_names=class_names, digits=4))
cm = confusion_matrix(y_val, y_val_pred)
print("Confusion matrix shape:", cm.shape)

# ---------- save artifacts ----------
import joblib
joblib.dump({'scaler': scaler if X_num.shape[1]>0 else None, 'cat_maps': cat_maps, 'use_sbert': use_sbert, 'sbert_name': (SBERT_NAME if use_sbert else None)}, os.path.join(MODEL_DIR,'preproc.joblib'))
model.save(os.path.join(MODEL_DIR,'final_model'), include_optimizer=False)
print("Saved model and preprocessing to", MODEL_DIR)

Classes: ['Benign', 'DDoS', 'DoS', 'MQTT', 'Reconnaissance', 'Spoofing'] n_classes: 6
Train size: 144000 Val size: 36000
num_in (144000, 45)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ num_in (InputLayer) │ (None, 45)        │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 256)       │     11,776 │ num_in[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, 256)       │        512 │ dense[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 256)       │          0 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 256)       │     65,792 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 256)       │        512 │ dense_1[0][0]     │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 256)       │          0 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 256)       │     65,792 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 256)       │        512 │ dense_2[0][0]     │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 256)       │          0 │ dropout[0][0],    │
│                     │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 256)       │     65,792 │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 256)       │        512 │ dense_3[0][0]     │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 256)       │          0 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 256)       │     65,792 │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 256)       │        512 │ dense_4[0][0]     │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 256)       │          0 │ add[0][0],        │
│                     │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 128)       │     32,896 │ add_1[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 512)       │     66,048 │ dense_5[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 512)       │      1,024 │ dense_6[0][0]     │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 512)       │          0 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 256)       │    131,328 │ dropout_3[0][0] 

 Total params: 510,342 (1.95 MB)

 Trainable params: 510,342 (1.95 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/80


InvalidArgumentError: Graph execution error:

Detected at node gradient_tape/compile_loss/loss/mul_1/BroadcastGradientArgs defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "d:\Prog\iomt-anomalyze\venv\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>

  File "d:\Prog\iomt-anomalyze\venv\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance

  File "d:\Prog\iomt-anomalyze\venv\Lib\site-packages\ipykernel\kernelapp.py", line 739, in start

  File "d:\Prog\iomt-anomalyze\venv\Lib\site-packages\tornado\platform\asyncio.py", line 211, in start

  File "C:\Python313\Lib\asyncio\base_events.py", line 683, in run_forever

  File "C:\Python313\Lib\asyncio\base_events.py", line 2042, in _run_once

  File "C:\Python313\Lib\asyncio\events.py", line 89, in _run

  File "d:\Prog\iomt-anomalyze\venv\Lib\site-packages\ipykernel\kernelbase.py", line 519, in dispa